In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [2]:
@numba.njit
def dynamicHardDisks(
        numberOfInitCycles: int, 
        numberOfProdCycles: int, 
        numberOfParticles: int, 
        maxDisplacement: float, 
        sampleFrequency: int,
        boxSize: float, 
    ):
    
    
    latticeSites = int(np.ceil(np.sqrt(numberOfParticles)))
    positions = np.zeros((latticeSites**2, 2))
    
    count = 0
    for x in np.linspace(0, (1-1/latticeSites) * boxSize, latticeSites):
        for y in np.linspace(0, (1-1/latticeSites) * boxSize, latticeSites):
            positions[count] = np.array([x,y])
            count += 1
    positions = positions[np.random.choice(numberOfParticles, numberOfParticles, replace=False)[:numberOfParticles]]
    
    
    samplePositions = np.zeros((numberOfProdCycles // sampleFrequency, numberOfParticles, 2))
    samplePositions[0] = positions
    sampleCounter = 1
    
    numberOfAttemptedMoves = 0
    numberOfAcceptedMoves = 0
    
    for cycle in range(numberOfInitCycles + numberOfProdCycles):
        numberOfAttemptedMoves += 1
        displacement = (np.random.rand(2) - 0.5) * maxDisplacement
        particleIdx = np.random.choice(numberOfParticles)
        newPosition = positions[particleIdx] + displacement
        
        overlap = False
        for k in range(numberOfParticles):
            if k == particleIdx:
                continue
            dr = newPosition - positions[k]
            dr = (dr + 0.5 * boxSize) % boxSize - 0.5 * boxSize
            r2 = np.linalg.norm(dr)
            
            if r2 < 1.0:
                overlap = True
        
        if not overlap:
            positions[particleIdx] = newPosition
            numberOfAcceptedMoves += 1
                
        if cycle >= numberOfInitCycles and cycle % sampleFrequency == 0:
            samplePositions[sampleCounter] = positions
            sampleCounter += 1
    print(numberOfAcceptedMoves/numberOfAttemptedMoves)
    return samplePositions % boxSize

In [3]:
x = dynamicHardDisks(
        numberOfInitCycles = int(1e6), 
        numberOfProdCycles = int(2e5), 
        numberOfParticles = 50, 
        maxDisplacement = 0.2, 
        sampleFrequency = 20000,
        boxSize = 10.0, 
    )

0.8448833333333333


: 

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
for i in range(3):
    ax.scatter(*x[i].T % 12.0, s=400)